# MAPPING CN WITH PN

### IMPORT

In [139]:
import pandas as pd
import polars as pl

### READ

In [140]:
cn = pd.read_csv("cn_data.csv")
pn = pd.read_csv("pn_data.csv")
prod = pl.read_csv(r"C:\WorkSpace\Python\DA_basics\Data\products.csv", infer_schema_length=10000,truncate_ragged_lines=True)

In [141]:
# If CN for Nov & Dec


#  li = ["Nov-2024","Dec-2024"]
# cn = cn[cn["month"].isin(li)]

### ASSIGNING UNIQUE KEYS

In [142]:
cn = cn.sort_values(by=["key", "vendor_cn_date"]).reset_index(drop=True)
pn = pn.sort_values(by=["key", "date"]).reset_index(drop=True)

unique_keys = pd.concat([cn["key"], pn["key"]]).unique() 
key_mapping = {key: idx + 1 for idx, key in enumerate(sorted(unique_keys))}

cn["unique_value"] = cn["key"].map(key_mapping)
pn["unique_value"] = pn["key"].map(key_mapping)

### GENERATING MAPPED TABLE (temp_df)

In [145]:
columns = ["vendor_id", "ws_code", "batch_number", "product_name", "cn_number", "cn_date", "cn_ref_date", "cn_qty",
           "pn_number","pn_date", "pn_qty", "finalqty", "Note", "key"]
temp_df = pd.DataFrame(columns=columns)
Note = ["CN pending", "Excess CN received", "Exact Match"]

# Fetch data helper function
def fetch_data(cn, pn, i, j, k, l):
    if j == -1:
        cn = pd.DataFrame([[None] * 1], columns=["x"])
        pn = pn.iloc[i]
        key = pn["key"]
        k = pn['quantity']
    elif i == -1:
        cn = cn.iloc[j]
        key = cn['key']
        pn = pd.DataFrame([[" "] * 1], columns=["x"])
        k = cn['billed_quantity']
    else:
        cn = cn.iloc[j]
        pn = pn.iloc[i]
        key = pn["key"]
    note = Note[l]
    vendor_id, ws_code, batch_num = key.split("_")
    def safe_get(data, column, default=" "):
        return data[column] if column in data else default

    # Build the data list
    data = [
        vendor_id,
        ws_code,
        batch_num,
        prod.filter(pl.col("ws_code") == int(ws_code))[0,"product_name"],
        safe_get(cn, "debit_note_number", " "),
        safe_get(cn, "vendor_cn_date", " "),
        " ",
        safe_get(cn, "billed_quantity", " "),
        safe_get(cn, "pick_note_number", " "),
        safe_get(pn, "date", " "),
        safe_get(pn, "quantity", " "),
        k,
        note,
        key
    ]
    return data

# Process mapping logic (same as earlier logic with minor modifications)
for unique_value in cn["unique_value"].unique():
    cn_filter = cn[cn["unique_value"] == unique_value]
    cn_filter_key = cn_filter["billed_quantity"].values
    i, j = 0, 0
    if unique_value in pn["unique_value"].values:
        pn_filter = pn[pn["unique_value"] == unique_value]
        pn_filter_key = pn_filter["quantity"].values
        i, j = 0, 0
        while i < len(pn_filter_key) and j < len(cn_filter_key):
            k = pn_filter_key[i] - cn_filter_key[j]
            if k < 0:
                temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn_filter, i, j, k * -1, 1)
                cn_filter_key[j] = k * -1
                i += 1
            elif k > 0:
                temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn_filter, i, j, k, 0)
                pn_filter_key[i] = k
                j += 1
            else:
                temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn_filter, i, j, k, 2)
                i += 1
                j += 1

        if len(pn_filter_key) > i or len(cn_filter_key) > j:
            if len(pn_filter_key) > i:
                while(len(pn_filter_key) > i):
                    temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn_filter, i, -1, 1, 0)
                    i += 1
            else:
                while(len(cn_filter_key) > j):
                    temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn_filter, -1, j, 1, 1)
                    j += 1

    else:
        while j < len(cn_filter_key):
            temp_df.loc[len(temp_df)] = fetch_data(cn_filter, pn, -1, j, 1, 1)
            j += 1

for unique_value in pn["unique_value"].unique():
    if unique_value not in cn["unique_value"].values:
        pn_filter = pn[pn["unique_value"] == unique_value]
        pn_filter_key = pn_filter["quantity"].values
        i = 0
        while i < len(pn_filter_key):
            temp_df.loc[len(temp_df)] = fetch_data(None, pn_filter, i, -1, pn_filter_key[i], 0)
            i += 1

### SAVE

In [146]:
temp_df.to_csv('final', index=False)